In [1]:
import pandas as pd
import os
import datetime
import plotly.express as px

In [2]:
airbnb_path = 'airbnb_.csv'
raw_df = pd.read_csv(airbnb_path)

In [3]:
raw_df.head(3)

,Date,Arriving by date,Type,Booking date,Start date,End date,Nights,Listing,Currency,Amount,Paid out,Host service fee,Fast pay fee,Cleaning fee,Pet fee,Gross earnings,Occupancy taxes,Earnings year
0,02/24/2024,02/24/2024,Payout,NaN,NaN,NaN,NaN,NaN,USD,NaN,950.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,02/24/2024,NaN,Reservation,02/21/2024,02/23/2024,02/25/2024,2.0,Treetops by ARRAY | 3 King | 7 Acre | Private ...,USD,435.53,NaN,13.47,NaN,130.0,0.0,449.0,67.89,2024.0
2,02/24/2024,NaN,Reservation,02/18/2024,02/23/2024,02/25/2024,2.0,"NEW Idlewood, Close to NCSU, PNC Arena, Fenced",USD,515.07,NaN,15.93,NaN,130.0,0.0,531.0,80.29,2024.0


In [4]:
def clean_airbnb_data(df):
    df = df.dropna(subset=['Listing'])
    df = df.drop(columns = ['Arriving by date', 'Currency', 'Paid out', 'Fast pay fee'])
    return df
cleaned_df = clean_airbnb_data(raw_df)

##### Lead Times

In [5]:
def create_lead_times_column(df):
    df['Start date'] = pd.to_datetime(df['Start date'])
    df['Booking date'] = pd.to_datetime(df['Booking date'])
    df['Lead time'] = (df['Booking date'] - df['Start date']).dt.days
    df['Lead time'] = df['Lead time'].abs()
    return df

In [6]:
df = create_lead_times_column(cleaned_df)

In [7]:
df.head(3)

,Date,Type,Booking date,Start date,End date,Nights,Listing,Amount,Host service fee,Cleaning fee,Pet fee,Gross earnings,Occupancy taxes,Earnings year,Lead time
1,02/24/2024,Reservation,2024-02-21,2024-02-23,02/25/2024,2.0,Treetops by ARRAY | 3 King | 7 Acre | Private ...,435.53,13.47,130.0,0.0,449.0,67.89,2024.0,2
2,02/24/2024,Reservation,2024-02-18,2024-02-23,02/25/2024,2.0,"NEW Idlewood, Close to NCSU, PNC Arena, Fenced",515.07,15.93,130.0,0.0,531.0,80.29,2024.0,5
4,02/22/2024,Reservation,2024-02-20,2024-02-21,02/23/2024,2.0,Treetops by ARRAY | 3 King | 7 Acre | Private ...,322.04,9.96,130.0,0.0,332.0,50.20,2024.0,1


##### Grouping

In [8]:
def monthly_kpis(df):
    df['Month'] = df['Start date'].dt.strftime('%B')
    df['Earnings year'] = df['Earnings year'].astype(int)

    result = df.groupby(['Earnings year', 'Month']).agg({
        'Nights': 'sum',
        'Lead time': 'mean',
        'Gross earnings': 'sum',
        'Host service fee': 'sum',
        'Cleaning fee': 'sum',
        'Pet fee': 'sum',
        'Occupancy taxes': 'sum'
    }).reset_index()

    result[['Lead time', 'Nights']] = result[['Lead time', 'Nights']].astype(int)

    month_order = ['January', 'February', 'March', 
                'April', 'May', 'June', 'July', 
                'August', 'September', 'October', 
                'November', 'December']
    result['Month'] = pd.Categorical(result['Month'], categories=month_order, ordered=True)
    result = result.sort_values(by=['Earnings year', 'Month'])

    result['Avg nightly price'] = round((result['Gross earnings'] / result['Nights']),2)

    result['Date'] = pd.to_datetime(result['Earnings year'].astype(str) + ' ' + result['Month'].astype(str), format='%Y %B')

    return result

In [9]:
result = monthly_kpis(df)

In [ ]:
# Create new columns for month and year
df['Month'] = df['Start date'].dt.strftime('%B')

In [ ]:
df['Earnings year'] = df['Earnings year'].astype(int)

In [ ]:
result = df.groupby(['Earnings year', 'Month']).agg({
    'Nights': 'sum',
    'Lead time': 'mean',
    'Gross earnings': 'sum',
    'Host service fee': 'sum',
    'Cleaning fee': 'sum',
    'Pet fee': 'sum',
    'Occupancy taxes': 'sum'
}).reset_index()

result[['Lead time', 'Nights']] = result[['Lead time', 'Nights']].astype(int)


In [ ]:
result

In [ ]:
month_order = ['January', 'February', 'March', 
               'April', 'May', 'June', 'July', 
               'August', 'September', 'October', 
               'November', 'December']
result['Month'] = pd.Categorical(result['Month'], categories=month_order, ordered=True)
result = result.sort_values(by=['Earnings year', 'Month'])

In [ ]:
result

In [ ]:
result['Avg nightly price'] = round((result['Gross earnings'] / result['Nights']),2)

In [ ]:
result['Date'] = pd.to_datetime(result['Earnings year'].astype(str) + ' ' + result['Month'].astype(str), format='%Y %B')

In [ ]:
result

In [ ]:
lead_time_fig = px.line(result, x='Date', y='Lead time', title='Lead Time')

In [ ]:
lead_time_fig

In [ ]:
gross_earnings_fig = px.line(result, x='Date', y='Gross earnings', title='Gross Earnings')

In [ ]:
gross_earnings_fig

In [ ]:
booked_nights_fig = px.line(result, x='Date', y='Nights', title='Booked Nights')

In [ ]:
booked_nights_fig

In [ ]:
avg_nightly_price_fig = px.line(result, x='Date', y='Avg nightly price', title='Average Nightly Price')

In [ ]:
avg_nightly_price_fig